## Tujuan

Dalam dunia bisnis, terdapat istilah Acquisition Cost dan Retention Cost. Acquisition Cost adalah biaya yang dikeluarkan oleh perusahaan untuk mendapatkan pembeli atau pelanggan baru. Sedangkan Retention Cost adalah biaya yang dikeluarkan oleh perusahaan untuk mempertahankan pelanggan yang ada.

Pada kenyataannya, kita sebagai manusia memiliki keterbatasan dalam memprediksi pelanggan mana yang akan beralih dari produk kita dan mana pelanggan yang akan tetap bertahan. Menurut beberapa sumber, besaran Acquisition Cost 5x lebih besar daripada Retention Cost. Jika kita salah memprediksi pelanggan mana yang sebenarnya akan churn, namun kita prediksi ia akan tetap bertahan, maka biaya yang dikeluarkan menjadi lebih besar.


1. Problem Statement for Machine Learning:
    1. Berapa banyak keuntungan yang hilang karena pelanggan churn? 
    1. Model apa yang paling tepat untuk mendeteksi pelanggan yang akan churn?
    1. Bagaimana memprediksi pelanggan yang churn dengan baik, sehingga dapat meminimalisasi prediksi yang berupa false negatif?
    1. Matriks apa yang akan digunakan untuk pengukuran kualitas machine learning?


2. Problem Statement for Analytics:
    1. Customer seperti apa yang paling banyak churn?
    1. Apa yang membuat customer bertahan menggunakan provider? Produk unggulan nya apa?
    1. Layanan apa yang paling tidak diminati oleh customer?

## Data Understanding

### 1. Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### 2. Data

In [2]:
telco = pd.read_csv('telco.csv')

In [3]:
telco.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
telco.tail()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


In [5]:
# Melihat tipe data pada setiap kolom

telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [6]:
# Melihat jumlah kolom dan baris yang dimiliki data

telco.shape

(7043, 21)

In [7]:
# Melihat apakah ada data ganda atau duplikat

print ('Jumlah data duplikat =', telco.duplicated().sum())

Jumlah data duplikat = 0


### Penjelasan Dataset
Dataset yang ada meliputi `7043` pengamatan dengan `20` fitur dan 1 label (`Churn`)

| No | __Nama Fitur__ | __Penjelasan__ | __Data Type__ |
| - | - | - | - | 
| 1 | CustomerID | Berisi ID unik setiap pelanggan | categorical | 
| 2 | Gender | Apakah pelanggan pria atau wanita | categorical |
| 3 | SeniorCitizen | Apakah pelanggan berusia di atas 65 tahun atau tidak (1, 0) | numeric, int |
| 4 | Partner | Apakah pelanggan memiliki pasangan atau tidak (Yes, No) | categorical |
| 5 | Dependents | Apakah pelanggan memiliki tanggungan atau tidak (Yes, No) | categorical | 
| 6 | Tenure | Jumlah bulan berlangganan pada perusahaan | numeric, int |
| 7 | PhoneService | Apakah pelanggan memiliki layanan telepon atau tidak (Yes, No) | categorical |
| 8 | MultipleLines | Apakah pelanggan menggunakan layanan multiple lines atau tidak (Yes, No, No phone service) | categorical |
| 9 | InternetService | Internet service provider yang digunakan pelanggan (DSL, Fiber optic, No) | categorical |
| 10 | OnlineSecurity | Apakah pelanggan memiliki layanan online security atau tidak (Yes, No, No internet service) | categorical | 
| 11 | OnlineBackup |  Apakah pelanggan memiliki layanan online back up atau tidak (Yes, No, No internet service) | categorical | 
| 12 | DeviceProtection | Apakah pelanggan memiliki layanan device protection atau tidak (Yes, No, No internet service) | categorical |
| 13 | TechSupport | Apakah pelanggan memiliki layanan tech support atau tidak (Yes, No, No internet service) | categorical | 
| 14 | StreamingTV | Apakah pelanggan menggunakan internet untuk streaming TV dari pihak ketiga atau tidak (Yes, No, No internet service) | categorical |
| 15 | StreamingMovies | Apakah pelanggan menggunakan internet untuk streaming film dari pihak ketiga atau tidak (Yes, No, No internet service) | categorical |
| 16 | Contract | Durasi kontrak pembayaran dari pelanggan (Month-to-month, One year, Two year) | categorical |
| 17 | PaperlessBilling | Apakah pelanggan menggunakan paperless billing atau tidak (Yes, No) | categorical |
| 18 | PaymentMethod | Metode pembayaran pelanggan (Electronic check, Mailed check, Bank transfer, Credit card) | categorical |
| 19 | MonthlyCharges | Jumlah tagihan bulanan dari pelanggan |  numeric , int |
| 20 | TotalCharges | Total keseluruhan tagihan dari pelanggan | object |
| 21 | Churn | Apakah pelanggan churn/beralih atau tidak (Yes or No) | categorical |

### 3. About the Data

* Target Feature: Churn
* Numeric Features: Tenure, MonthlyCharges, and TotalCharges
* Categorical Features: CustomerID, Gender, SeniorCitizen, Partner, Dependents, PhoneService, MulitpleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies, Contract, PaperlessBilling, PaymentMethod

### 4. Statistical Summary

In [8]:
nums = ['SeniorCitizen', 'tenure', 'MonthlyCharges']
cats = ['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
        'Contract', 'PaperlessBilling', 'PaymentMethod', 'TotalCharges', 'Churn']

In [9]:
telco[nums].describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [10]:
telco[cats].describe()

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,1167-OYZJF,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [11]:
# Melihat apakah ada data yang kosong atau tidak

telco.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [12]:
telco_items = []
for col in telco.columns:
    telco_items.append([col, telco[col].dtype, telco[col].isna().sum(), round((telco[col].isna().sum()/len(telco[col]))*100,2),
                      telco[col].nunique(), list(telco[col].sample(30).drop_duplicates().values)])

telcoDesc = pd.DataFrame(columns=['feature', 'type_data', 'null', '%null', 'unique', 'uniqueSample'],data=telco_items)
telcoDesc

,feature,type_data,null,%null,unique,uniqueSample
0,customerID,object,0,0.0,7043,"[5774-XZTQC, 9776-OJUZI, 3063-QFSZL, 1322-AGOQ..."
1,gender,object,0,0.0,2,"[Female, Male]"
2,SeniorCitizen,int64,0,0.0,2,"[0, 1]"
3,Partner,object,0,0.0,2,"[No, Yes]"
4,Dependents,object,0,0.0,2,"[No, Yes]"
5,tenure,int64,0,0.0,73,"[19, 25, 72, 15, 52, 6, 1, 16, 8, 54, 65, 53, ..."
6,PhoneService,object,0,0.0,2,"[Yes, No]"
7,MultipleLines,object,0,0.0,3,"[Yes, No, No phone service]"
8,InternetService,object,0,0.0,3,"[DSL, No, Fiber optic]"
9,OnlineSecurity,object,0,0.0,3,"[No, Yes, No internet service]"


### 5. Data Reshaping

Sebagaimana kita lihat, bahwa tidak ada missing value. Tapi kolom ‘Total Charges’ seharusnya bertipe numerik.

In [13]:
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'], errors='coerce')
telco['TotalCharges'].dtype

dtype('float64')

Mengubah kolom string menjadi numerik dapat menyebabkan missing value. 

In [14]:
telco['TotalCharges'].isnull().sum()

11

Terdapat 11 missing value, tapi bisa saja hal tersebut diakibatkan karena memang tidak ada biaya pada pelanggan tersebut. Bisa kita cari tahu di kolom tenure dan membandingkan indexnya.

In [15]:
telco['tenure'].isin([0]).sum()

11

In [16]:
print(telco[telco['tenure'].isin([0])].index)
print(telco[telco['TotalCharges'].isnull()].index)

Int64Index([488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754], dtype='int64')
Int64Index([488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754], dtype='int64')


Index pada kolom __Total Charges yang bernilai 0 (nol)__ sama dengan index pada kolom __Tenure yang juga bernilai 0 (nol)__. Hal in mengindikasikan bahwa missing value terjadi karena memang 11 pelanggan tersebut churn sebelum kurun waktu 1 bulan berlangganan. Maka missing value tersebut dapat kita isi dengan angka 0 (nol).

In [17]:
telco.loc[:,'TotalCharges'] = telco.loc[:,'TotalCharges'].replace(np.nan,0)
telco['TotalCharges'].isnull().sum()

0

In [18]:
for col in telco.columns:
    print(col, '\n')
    print(telco[col].value_counts(), '\n')

customerID 

1167-OYZJF    1
5575-GNVDE    1
9474-PHLYD    1
8361-LBRDI    1
2636-OHFMN    1
             ..
4789-KWMXN    1
0611-DFXKO    1
4456-RHSNB    1
9819-FBNSV    1
8085-MSNLK    1
Name: customerID, Length: 7043, dtype: int64 

gender 

Male      3555
Female    3488
Name: gender, dtype: int64 

SeniorCitizen 

0    5901
1    1142
Name: SeniorCitizen, dtype: int64 

Partner 

No     3641
Yes    3402
Name: Partner, dtype: int64 

Dependents 

No     4933
Yes    2110
Name: Dependents, dtype: int64 

tenure 

1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64 

PhoneService 

Yes    6361
No      682
Name: PhoneService, dtype: int64 

MultipleLines 

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64 

InternetService 

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64 

OnlineSecurit

### 6. Data Cleaning

In [19]:
telco.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [20]:
# Ubah nama fitur 'tenure' dan 'gender' agar seragam dengan yang lain

telco = telco.rename(columns={'tenure': 'Tenure', 'gender': 'Gender'})

In [21]:
telco.columns

Index(['customerID', 'Gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'Tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [22]:
# Drop kolom customerID, karena tidak akan dibutuhkan

telco = telco.drop(['customerID'], axis = 1)

In [23]:
# mengubah value dari kolom senior citizen agar lebih mudah dibaca

telco["SeniorCitizen"]= telco["SeniorCitizen"].map({0: "No", 1: "Yes"})

In [24]:
telco.head()

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,No,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,No,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Export ke csv

In [25]:
telco.to_csv('telco_clean.csv',index=False)